In [1]:
import MetaTrader5 as mt
import pandas as pd
import plotly.express as px
import matplotlib.pylab as plt
import numpy as np
import talib
from talipp.indicators import EMA, SMA, Stoch, DPO
from joblib import dump
from datetime import datetime
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, confusion_matrix, classification_report
from own_functions import *
import os
from tsfresh import extract_features, select_features
from tsfresh.utilities.dataframe_functions import roll_time_series, make_forecasting_frame
from tsfresh.utilities.dataframe_functions import impute

In [2]:
mt.initialize()
login = 51708234
password ="4bM&wuVJcBTnjV"
server = "ICMarketsEU-Demo"
mt.login(login,password,server)

True

In [3]:
symbol = "USDCAD"
timeframe = mt.TIMEFRAME_D1
ohlc_data = pd.DataFrame(mt.copy_rates_range(symbol, timeframe, datetime(2010, 1, 1), datetime(2023, 12, 31)))
ohlc_data['time'] = pd.to_datetime(ohlc_data['time'], unit='s')
df = ohlc_data[['time', 'open', 'high', 'low', 'close']].copy()

# Indicators
df['WILLR_15'] = talib.WILLR(df['high'], df['low'], df['close'], timeperiod=15)
df['WILLR_23'] = talib.WILLR(df['high'], df['low'], df['close'], timeperiod=23)
df['WILLR_42'] = talib.WILLR(df['high'], df['low'], df['close'], timeperiod=42)
df['WILLR_145'] = talib.WILLR(df['high'], df['low'], df['close'], timeperiod=145)

# Buy & Sell Flags
df['b_flag'] = 0
df['s_flag'] = 0

# Dropping NaN values and resetting index
df = df.dropna().reset_index(drop=True)

""" csv_file_path = 'ICMT5EURUSD2010_3112_D1.csv'  # Specify your desired path
df.to_csv(csv_file_path, index=False)
df """

" csv_file_path = 'ICMT5EURUSD2010_3112_D1.csv'  # Specify your desired path\ndf.to_csv(csv_file_path, index=False)\ndf "

In [4]:
StopLoss = 1
TakeProfit = 2
BreakEvenRatio=StopLoss/(StopLoss+TakeProfit)
label_data(df,[StopLoss],[TakeProfit],80,symbol,False)
#print('BreatEvenRatio:', BreakEvenRatio)

Mean Candle: 0.008802622527944972


In [5]:
# Calculate total number of 1s in b_flag and s_flag columns
total_b_flags = df['b_flag'].sum()
total_s_flags = df['s_flag'].sum()

# Total number of rows in the DataFrame
total_rows = len(df)

# Calculate counts in segments of complete 100% data
count_100_b_flags = total_b_flags
count_100_s_flags = total_s_flags

# Calculate counts in intervals of 10%
interval_counts = []
for i in range(0, 101, 10):
    start_idx = int(i / 100 * total_rows)
    end_idx = int((i + 10) / 100 * total_rows)
    
    interval_b_flags = df['b_flag'].iloc[start_idx:end_idx].sum()
    interval_s_flags = df['s_flag'].iloc[start_idx:end_idx].sum()
    
    interval_counts.append((f'{i}% - {i+10}%', interval_b_flags, interval_s_flags))

# Print results
print("Total number of 1s:")
print(f"b_flag: {total_b_flags}")
print(f"s_flag: {total_s_flags}")

print("\nCounts in segments of 100% data:")
print(f"b_flag: {count_100_b_flags}")
print(f"s_flag: {count_100_s_flags}")

print("\nCounts in intervals of 10%:")
for interval, count_b, count_s in interval_counts:
    print(f"{interval}: b_flag={count_b}, s_flag={count_s}")

Total number of 1s:
b_flag: 1226
s_flag: 1000

Counts in segments of 100% data:
b_flag: 1226
s_flag: 1000

Counts in intervals of 10%:
0% - 10%: b_flag=105, s_flag=109
10% - 20%: b_flag=100, s_flag=127
20% - 30%: b_flag=155, s_flag=97
30% - 40%: b_flag=161, s_flag=63
40% - 50%: b_flag=107, s_flag=111
50% - 60%: b_flag=114, s_flag=115
60% - 70%: b_flag=117, s_flag=93
70% - 80%: b_flag=102, s_flag=139
80% - 90%: b_flag=163, s_flag=78
90% - 100%: b_flag=102, s_flag=68
100% - 110%: b_flag=0, s_flag=0


In [6]:
# Feature extraction

df.drop(columns=['b_flag'], inplace=True)

selected_signal_1 = 'WILLR_15'
df_melted_1 = df[['time', selected_signal_1]].copy()
df_melted_1["Symbols"] = symbol

df_rolled_1 = roll_time_series(df_melted_1, column_id="Symbols", column_sort="time",
                               max_timeshift=20, min_timeshift=5)

X1 = extract_features(df_rolled_1.drop("Symbols", axis=1), 
                      column_id="id", column_sort="time", column_value=selected_signal_1, 
                      impute_function=impute, show_warnings=False)

X1 = X1.set_index(X1.index.map(lambda x: x[1]), drop=True)
X1.index.name = "time"
X1 = X1.dropna()

selected_signal_2 = 'WILLR_42'
df_melted_2 = df[['time', selected_signal_2]].copy()
df_melted_2["Symbols"] = symbol

df_rolled_2 = roll_time_series(df_melted_2, column_id="Symbols", column_sort="time",
                               max_timeshift=20, min_timeshift=5)

X2 = extract_features(df_rolled_2.drop("Symbols", axis=1), 
                      column_id="id", column_sort="time", column_value=selected_signal_2, 
                      impute_function=impute, show_warnings=False)

X2 = X2.set_index(X2.index.map(lambda x: x[1]), drop=True)
X2.index.name = "time"
X2 = X2.dropna()

X = pd.concat([X1, X2], axis=1, join='inner')
X = X.dropna()

# Align indices
df['time'] = pd.to_datetime(df['time'])
df = df.set_index('time')
df = df[df.index.isin(X.index)]

X = pd.concat([X, df], axis=1, join='inner')

# Ensure b_flag is at the end after feature selection
X_df = select_features(X, X['s_flag'])
X_df = X_df[[col for col in X_df if col != 's_flag'] + ['s_flag']]

# Get the list of selected feature names
selected_feature_names_X = list(X_df.columns)

# Combine lists if you need a single list for all selected features

print(selected_feature_names_X )

Feature Extraction: 100%|██████████| 30/30 [01:23<00:00,  2.78s/it]


['WILLR_42__mean_second_derivative_central', 'WILLR_15__mean_second_derivative_central', 'WILLR_15', 'WILLR_42__fft_coefficient__attr_"imag"__coeff_7', 'WILLR_15__fft_coefficient__attr_"imag"__coeff_7', 'WILLR_23', 'WILLR_15__fft_coefficient__attr_"real"__coeff_10', 'WILLR_42__fft_coefficient__attr_"real"__coeff_10', 'WILLR_42__fft_coefficient__attr_"imag"__coeff_6', 'WILLR_42__fft_coefficient__attr_"imag"__coeff_8', 'WILLR_15__fft_coefficient__attr_"imag"__coeff_6', 'WILLR_15__fft_coefficient__attr_"real"__coeff_9', 'WILLR_15__fft_coefficient__attr_"imag"__coeff_8', 'WILLR_42', 'WILLR_15__agg_linear_trend__attr_"slope"__chunk_len_10__f_agg_"max"', 'WILLR_42__fft_coefficient__attr_"real"__coeff_9', 'WILLR_42__agg_linear_trend__attr_"rvalue"__chunk_len_10__f_agg_"mean"', 'WILLR_42__fft_coefficient__attr_"angle"__coeff_7', 'WILLR_15__agg_linear_trend__attr_"stderr"__chunk_len_5__f_agg_"max"', 'WILLR_15__agg_linear_trend__attr_"slope"__chunk_len_10__f_agg_"mean"', 'WILLR_42__fft_coefficie

In [66]:
sum_fp = 0
sum_tp = 0

split = int(0.80 * len(X_df))

train_data, test_data = X_df.iloc[:split], X_df.iloc[split:]

# Train data
x_train = train_data.iloc[:, :-1].values
y_train = train_data['s_flag'].values
# Test data
x_test = test_data.iloc[:, :-1].values
y_test = test_data['s_flag'].values

# Scale Data
sc_mt = StandardScaler()
x_train = sc_mt.fit_transform(x_train)
x_test = sc_mt.transform(x_test)

os.makedirs('Dump_GBPUSD_D1_3112_Sell', exist_ok=True)
dump(sc_mt, 'Dump_GBPUSD_D1_3112_Sell/scaler.joblib')

# Hyperparameter
n_estimators = 180
class_weight = {0: 6, 1: 1}
max_features = 'sqrt'
random_state = 1

# Initialise RandomForestClassifier
rf_classifier_mt = RandomForestClassifier(
    n_estimators=n_estimators,
    class_weight=class_weight,
    max_features=max_features,
    random_state=random_state
)

# Train Model
rf_classifier_mt.fit(x_train, y_train)

dump(rf_classifier_mt, 'Dump_GBPUSD_D1_3112_Sell/model.joblib')

# Predict
y_pred = rf_classifier_mt.predict(x_test)

print("Confusion Matrix:")
cm = confusion_matrix(y_test, y_pred)
print(cm)

false_positives = cm[0][1]
true_positives = cm[1][1]

sum_fp += false_positives
sum_tp += true_positives

precision = precision_score(y_test, y_pred)

print('WIN/LOSS-Diff:', round(100 * (precision - BreakEvenRatio), 2), '%')
print('sum_fp:', sum_fp)
print('sum_tp:', sum_tp)
print('precision:', precision)
print('Ratio total:', round(100 * (sum_tp / (sum_fp + sum_tp)), 2))
print('BreakEvenRatio:', round(BreakEvenRatio, 2))
print('____________________________________________________________________________________________________________________________')

Confusion Matrix:
[[432 120]
 [ 74  71]]
WIN/LOSS-Diff: 3.84 %
sum_fp: 120
sum_tp: 71
precision: 0.3717277486910995
Ratio total: 37.17
BreakEvenRatio: 0.33
____________________________________________________________________________________________________________________________


In [8]:
df_pred = pd.DataFrame(index=test_data.index)
df_pred['prediction'] = y_pred
df_pred.to_csv('predictGBPUSD_D1_3112_Sell.csv')

In [9]:
symbol = "GBPUSD"
timeframe = mt.TIMEFRAME_D1
ohlc_data = pd.DataFrame(mt.copy_rates_range(symbol, timeframe, datetime(2010, 1, 1), datetime(2023, 12, 31)))
ohlc_data['time'] = pd.to_datetime(ohlc_data['time'], unit='s')
df = ohlc_data[['time', 'open', 'high', 'low', 'close']].copy()

# Indicators
df['WILLR_15'] = talib.WILLR(df['high'], df['low'], df['close'], timeperiod=15)
df['WILLR_23'] = talib.WILLR(df['high'], df['low'], df['close'], timeperiod=23)
df['WILLR_42'] = talib.WILLR(df['high'], df['low'], df['close'], timeperiod=42)
df['WILLR_145'] = talib.WILLR(df['high'], df['low'], df['close'], timeperiod=145)

# Buy & Sell Flags
df['b_flag'] = 0
df['s_flag'] = 0

# Dropping NaN values and resetting index
df = df.dropna().reset_index(drop=True)


In [10]:
""" import os
from joblib import dump
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, precision_score, make_scorer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from imblearn.over_sampling import SMOTE

# Assuming X_df is your dataframe

# Set test size percentage (e.g., 20% of the data)
test_size_percentage = 0.20
split = int((1 - test_size_percentage) * len(X_df))

train_data, test_data = X_df.iloc[:split], X_df.iloc[split:]

# Train data
x_train = train_data.iloc[:, :-1].values
y_train = train_data['s_flag'].values
# Test data
x_test = test_data.iloc[:, :-1].values
y_test = test_data['s_flag'].values

# Scale Data
sc_mt = StandardScaler()
x_train = sc_mt.fit_transform(x_train)
x_test = sc_mt.transform(x_test)

os.makedirs('Dump_GBPUSD_D1_3112_Sell', exist_ok=True)
dump(sc_mt, 'Dump_GBPUSD_D1_3112_Sell/scaler.joblib')

# Resampling using SMOTE
smote = SMOTE(random_state=8)
x_train_res, y_train_res = smote.fit_resample(x_train, y_train)

# Hyperparameter tuning using TimeSeriesSplit
param_grid = {
    'n_estimators': [100, 200, 300,400,500,600,700,800,900],
    'class_weight': [{0: 5, 1: 10}, {0: 3, 1: 7}],
    'max_features': ['auto', 'log2'],
    'random_state': [0,8,10,42,60,80,100,150]
}

# Custom scoring function to prioritize true positives and limit false positives
def custom_scorer(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    false_positives = cm[0][1]
    true_positives = cm[1][1]
    if true_positives >= 50 and false_positives < 15:
        return 1  # High score for models meeting the criteria
    else:
        return 0  # Low score for models not meeting the criteria

scorer = make_scorer(custom_scorer, greater_is_better=True)

# Use TimeSeriesSplit for cross-validation
tscv = TimeSeriesSplit(n_splits=5)

rf_classifier_mt = RandomForestClassifier()

grid_search = GridSearchCV(estimator=rf_classifier_mt, param_grid=param_grid, scoring=scorer, cv=tscv, verbose=1, n_jobs=-1)
grid_search.fit(x_train_res, y_train_res)

best_model = grid_search.best_estimator_

# Print the best model parameters
print("Best model parameters:")
print(grid_search.best_params_)

dump(best_model, 'Dump_GBPUSD_D1_3112_Sell/model.joblib')

# Predict
y_pred = best_model.predict(x_test)

print("Confusion Matrix:")
cm = confusion_matrix(y_test, y_pred)
print(cm)

false_positives = cm[0][1]
true_positives = cm[1][1]

sum_fp = false_positives
sum_tp = true_positives

precision = precision_score(y_test, y_pred)

print('WIN/LOSS-Diff:', round(100 * (precision - BreakEvenRatio), 2), '%')
print('sum_fp:', sum_fp)
print('sum_tp:', sum_tp)
print('precision:', precision)
print('Ratio total:', round(100 * (sum_tp / (sum_fp + sum_tp)), 2))
print('BreakEvenRatio:', round(BreakEvenRatio, 2))
print('____________________________________________________________________________________________________________________________')
 """

' import os\nfrom joblib import dump\nimport numpy as np\nimport pandas as pd\nfrom sklearn.ensemble import RandomForestClassifier\nfrom sklearn.metrics import confusion_matrix, precision_score, make_scorer\nfrom sklearn.preprocessing import StandardScaler\nfrom sklearn.model_selection import TimeSeriesSplit, GridSearchCV\nfrom imblearn.over_sampling import SMOTE\n\n# Assuming X_df is your dataframe\n\n# Set test size percentage (e.g., 20% of the data)\ntest_size_percentage = 0.20\nsplit = int((1 - test_size_percentage) * len(X_df))\n\ntrain_data, test_data = X_df.iloc[:split], X_df.iloc[split:]\n\n# Train data\nx_train = train_data.iloc[:, :-1].values\ny_train = train_data[\'s_flag\'].values\n# Test data\nx_test = test_data.iloc[:, :-1].values\ny_test = test_data[\'s_flag\'].values\n\n# Scale Data\nsc_mt = StandardScaler()\nx_train = sc_mt.fit_transform(x_train)\nx_test = sc_mt.transform(x_test)\n\nos.makedirs(\'Dump_GBPUSD_D1_3112_Sell\', exist_ok=True)\ndump(sc_mt, \'Dump_GBPUSD_D1_